# DSC 106 Midterm Project

In [1]:
#import packages
import spotipy
import spotipy.util as util
import pandas as pd
import urllib.request

In [2]:
#Spotify API authentication token. The first parameter is the Spotify user id.
token = util.prompt_for_user_token('b5jfei3kyhlyuiadni4zvzl84',
                                   client_id = 'bee2dc948f6b4124830cd17f29fcf864',
                                   client_secret = '9ff98c1bb35345fcb051369450c295f6',
                                   redirect_uri = 'http://localhost:8000/callback')
#access the Spotify API
sp = spotipy.Spotify(auth=token)

In [3]:
#access the playlist we made that contains our training tracks
training_playlist = sp.user_playlist('biggurlandyouknowit', 'spotify:playlist:1ZouLC1keHDTVeXRQDx6dq')

In [4]:
#basic info for the training tracks
training_tracks_df = pd.DataFrame()
for track_data in training_playlist['tracks']['items']:
    track_df = pd.DataFrame(pd.DataFrame(dict([(k,pd.Series(v)) for k,v in track_data.items()]))['track']).transpose()
    training_tracks_df = training_tracks_df.append(track_df)
training_tracks_df = training_tracks_df.drop(training_tracks_df.columns[0], axis = 1).reset_index(drop = True)
training_tracks_df

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track,track_number,type,uri,url
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,False,False,{'isrc': 'USPR38619998'},{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,37ZJ0p5Jm13JPevGcx4SkF,False,Livin' On A Prayer,83,None,True,3,track,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,NaN
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,False,False,{'isrc': 'GBCEE8000027'},{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,5HkFTCxSeJ3kGNyQJbT4rJ,False,Another One Bites The Dust,48,None,True,3,track,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,NaN
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,False,False,{'isrc': 'USGF18714801'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,0G21yYKMZoHa30cYVi1iA8,False,Welcome To The Jungle,72,None,True,1,track,spotify:track:0G21yYKMZoHa30cYVi1iA8,NaN
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,False,False,{'isrc': 'GBAAA8500001'},{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,3fH4KjXFYMmljxrcGrbPj9,False,Don't You (Forget About Me),75,None,True,1,track,spotify:track:3fH4KjXFYMmljxrcGrbPj9,NaN
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,False,False,{'isrc': 'USWB18300001'},{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,6Fba9RZtC6vTY814JToDtP,False,Jump,58,https://p.scdn.co/mp3-preview/5c4fca39cd964267...,True,7,track,spotify:track:6Fba9RZtC6vTY814JToDtP,NaN
5,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,False,False,{'isrc': 'GBCEE0500364'},{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,6l8GvAyoUZwWDgF1e4822w,False,Bohemian Rhapsody,61,None,True,9,track,spotify:track:6l8GvAyoUZwWDgF1e4822w,NaN
6,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,False,False,{'isrc': 'USSM10011897'},{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,5MxNLUsfh7uzROypsoO5qe,False,Dream On,78,https://p.scdn.co/mp3-preview/4c7fea33f70ea106...,True,3,track,spotify:track:5MxNLUsfh7uzROypsoO5qe,NaN
7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,False,False,{'isrc': 'USAT29900620'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,51pQ7vY7WXzxskwloaeqyj,False,Stairway to Heaven - 1990 Remaster,69,https://p.scdn.co/mp3-preview/da57b9497b855355...,True,4,track,spotify:track:51pQ7vY7WXzxskwloaeqyj,NaN
8,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,254453,False,False,{'isrc': 'USWB19900178'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lbXEepatjRV...,1lbXEepatjRVjoG8pZMtdp,False,Dreams,62,https://p.scdn.co/mp3-preview/10b5068313e5ee28...,True,9,track,spotify:track:1lbXEepatjRVjoG8pZMtdp,NaN
9,"{'album_type':

In [5]:
#audio features for the training tracks
training_audio_features_df = pd.DataFrame()
for index, row in training_tracks_df.iterrows():
    audio_feature = pd.DataFrame(sp.audio_features('spotify:track:' + row['id']))
    training_audio_features_df = training_audio_features_df.append(audio_feature)
training_audio_features_df = training_audio_features_df.reset_index(drop = True)
training_audio_features_df

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.077800,https://api.spotify.com/v1/audio-analysis/37ZJ...,0.532,249293,0.887,37ZJ0p5Jm13JPevGcx4SkF,0.000214,0,0.2940,-3.757,1,0.0335,122.511,4,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,audio_features,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,0.795
1,0.044400,https://api.spotify.com/v1/audio-analysis/5HkF...,0.922,215933,0.392,5HkFTCxSeJ3kGNyQJbT4rJ,0.165000,5,0.1230,-12.083,0,0.1220,109.878,4,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,audio_features,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,0.771
2,0.019500,https://api.spotify.com/v1/audio-analysis/0G21...,0.453,273480,0.987,0G21yYKMZoHa30cYVi1iA8,0.283000,8,0.2680,-4.485,1,0.0889,123.544,4,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,audio_features,spotify:track:0G21yYKMZoHa30cYVi1iA8,0.316
3,0.168000,https://api.spotify.com/v1/audio-analysis/3fH4...,0.660,263040,0.816,3fH4KjXFYMmljxrcGrbPj9,0.018100,2,0.0608,-6.610,1,0.0299,111.346,4,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,audio_features,spotify:track:3fH4KjXFYMmljxrcGrbPj9,0.678
4,0.074500,https://api.spotify.com/v1/audio-analysis/6Fba...,0.547,239560,0.822,6Fba9RZtC6vTY814JToDtP,0.009430,0,0.0645,-7.201,1,0.0292,130.138,4,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,audio_features,spotify:track:6Fba9RZtC6vTY814JToDtP,0.774
5,0.271000,https://api.spotify.com/v1/audio-analysis/6l8G...,0.397,355400,0.386,6l8GvAyoUZwWDgF1e4822w,0.000000,0,0.1880,-10.405,0,0.0503,144.242,4,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,audio_features,spotify:track:6l8GvAyoUZwWDgF1e4822w,0.210
6,0.388000,https://api.spotify.com/v1/audio-analysis/5MxN...,0.307,266960,0.433,5MxNLUsfh7uzROypsoO5qe,0.000091,1,0.3320,-10.057,1,0.0290,160.900,4,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,audio_features,spotify:track:5MxNLUsfh7uzROypsoO5qe,0.224
7,0.575000,https://api.spotify.com/v1/audio-analysis/51pQ...,0.346,478173,0.335,51pQ7vY7WXzxskwloaeqyj,0.006740,7,0.1850,-12.453,1,0.0339,84.204,4,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,audio_features,spotify:track:51pQ7vY7WXzxskwloaeqyj,0.213
8,0.139000,https://api.spotify.com/v1/audio-analysis/1lbX...,0.823,254453,0.338,1lbXEepatjRVjoG8pZMtdp,0.001290,0,0.5000,-15.899,1,0.0413,120.473,4,https://api.spotify.com/v1/tracks/1lbXEepatjRV...,audio_features,spotify:track:1lbXEepatjRVjoG8pZMtdp,0.785
9,0.667000,https://api.spotify.com/v1/audio-analysis/5xhP...,0.326,219440,0.365,5xhP99gVBIkpr7omcdCeiy,0.000523,3,0.0718,-8.776,1,0.0265,133.084,4,https://api.spotify.com/v1/tracks/5xhP99gVBIkp...,audio_features,spotify:track:5xhP99gVBIkpr7omcdCeiy,0.314


In [6]:
#training tracks with BOTH basic track info and audio features combined
training_df = training_tracks_df.merge(training_audio_features_df, on = 'id')
training_df

,album,artists,available_markets,disc_number,duration_ms_x,episode,explicit,external_ids,external_urls,href,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type_y,uri_y,valence
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,False,False,{'isrc': 'USPR38619998'},{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,...,0.2940,-3.757,1,0.0335,122.511,4,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,audio_features,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,0.795
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,False,False,{'isrc': 'GBCEE8000027'},{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,...,0.1230,-12.083,0,0.1220,109.878,4,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,audio_features,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,0.771
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,False,False,{'isrc': 'USGF18714801'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,...,0.2680,-4.485,1,0.0889,123.544,4,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,audio_features,spotify:track:0G21yYKMZoHa30cYVi1iA8,0.316
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,False,False,{'isrc': 'GBAAA8500001'},{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,...,0.0608,-6.610,1,0.0299,111.346,4,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,audio_features,spotify:track:3fH4KjXFYMmljxrcGrbPj9,0.678
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,False,False,{'isrc': 'USWB18300001'},{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,...,0.0645,-7.201,1,0.0292,130.138,4,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,audio_features,spotify:track:6Fba9RZtC6vTY814JToDtP,0.774
5,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,False,False,{'isrc': 'GBCEE0500364'},{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,...,0.1880,-10.405,0,0.0503,144.242,4,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,audio_features,spotify:track:6l8GvAyoUZwWDgF1e4822w,0.210
6,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,False,False,{'isrc': 'USSM10011897'},{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,...,0.3320,-10.057,1,0.0290,160.900,4,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,audio_features,spotify:track:5MxNLUsfh7uzROypsoO5qe,0.224
7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,False,False,{'isrc': 'USAT29900620'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,...,0.1850,-12.453,1,0.0339,84.204,4,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,audio_features,spotify:track:51pQ7vY7WXzxskwloaeqyj,0.213
8,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,254453,False,False,{'isrc': 'USWB19900178'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lbXEepatjRV...,.

In [7]:
#extract release date as a column
def getReleaseDate(x):
    return x['album']['release_date']
training_df['release_date'] = training_df.apply(getReleaseDate, axis = 1)

In [10]:
#create a decade column
def decade(x):
    year = int(x[0:4])
    if year >= 1950 and year < 1960:
        return '1950s'
    if year >= 1960 and year < 1970:
        return '1960s'
    if year >= 1970 and year < 1980:
        return '1970s'
    if year >= 1980 and year < 1990:
        return '1980s'
    if year >= 1990 and year < 2000:
        return '1990s'
    if year >= 2000 and year < 2010:
        return '2000s'
    if year >= 2010 and year < 2020:
        return '2010s'
    if year >= 2020 and year < 2030:
        return '2020s'
training_df['decade'] = training_df['release_date'].apply(decade)

In [11]:
training_df

,album,artists,available_markets,disc_number,duration_ms_x,episode,explicit,external_ids,external_urls,href,...,mode,speechiness,tempo,time_signature,track_href,type_y,uri_y,valence,release_date,decade
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,False,False,{'isrc': 'USPR38619998'},{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,...,1,0.0335,122.511,4,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,audio_features,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,0.795,1986-08-16,1980s
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,False,False,{'isrc': 'GBCEE8000027'},{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,...,0,0.1220,109.878,4,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,audio_features,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,0.771,1981-10-26,1980s
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,False,False,{'isrc': 'USGF18714801'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,...,1,0.0889,123.544,4,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,audio_features,spotify:track:0G21yYKMZoHa30cYVi1iA8,0.316,1987-07-21,1980s
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,False,False,{'isrc': 'GBAAA8500001'},{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,...,1,0.0299,111.346,4,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,audio_features,spotify:track:3fH4KjXFYMmljxrcGrbPj9,0.678,1985,1980s
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,False,False,{'isrc': 'USWB18300001'},{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,...,1,0.0292,130.138,4,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,audio_features,spotify:track:6Fba9RZtC6vTY814JToDtP,0.774,1996-09-24,1990s
5,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,False,False,{'isrc': 'GBCEE0500364'},{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,...,0,0.0503,144.242,4,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,audio_features,spotify:track:6l8GvAyoUZwWDgF1e4822w,0.210,2006-01-01,2000s
6,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,False,False,{'isrc': 'USSM10011897'},{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,...,1,0.0290,160.900,4,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,audio_features,spotify:track:5MxNLUsfh7uzROypsoO5qe,0.224,1973-01-05,1970s
7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,False,False,{'isrc': 'USAT29900620'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,...,1,0.0339,84.204,4,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,audio_features,spotify:track:51pQ7vY7WXzxskwloaeqyj,0.213,1971-11-08,1970s
8,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,254453,False,False,{'isrc': 'USWB19900178'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1